Best parameters: {'n_estimators': 400, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 7, 'learning_rate': 0.05} 
RMSE: 7.27829388170363

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer
from sklearn.model_selection import train_test_split

In [10]:
df_pred=pd.read_csv("df_cc.csv")
df_pred.drop('Unnamed: 0', axis=1, inplace=True)
df_pred

,id,device_id,active_power,direct_power,charge_capacity,date_time,T,U,Ff,RRR,...,T_b,U_b,Ff_b,RRR_b,DD_WE_b,DD_NS_b,Po(p)_b,POA_b,difference,hour
0,104983,6,2.31,2.69,1.8,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
1,104984,7,2.38,2.79,1.9,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
2,104985,8,2.37,2.83,1.9,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
3,104986,9,2.44,2.84,2.2,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
4,104987,10,2.32,2.68,1.8,2024-02-23 08:00:00,5.2,95,1,17.0,...,5.8,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,106183,6,3.70,4.24,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,9.6,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8
251,106184,7,3.80,3.99,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,9.6,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8
252,106185,8,3.80,4.19,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,9.6,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8
253,106186,9,4.02,4.39,0.8,2024-03-04 08:00:00,9.6,88,3,0.5,...,9.6,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8


In [11]:
features = ['T', 'U', 'Ff',	'RRR', 'DD_WE', 'DD_NS', 'POA', 'Po(p)', 'hour',
            'T_b', 'U_b', 'Ff_b', 'RRR_b', 'DD_WE_b', 'DD_NS_b', 'POA_b', 'Po(p)_b', 'device_id']

X_pred = df_pred[features]
y_pred = df_pred['cc_diff']

X_pred = X_pred.fillna(X_pred.mean())
y_pred = y_pred.fillna(y_pred.mean())

In [12]:
df=pd.read_csv("../models_cc_data/df_cc_init.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,id,device_id,active_power,direct_power,charge_capacity,T,U,Ff,RRR,DD_WE,...,active_power_b,T_b,U_b,Ff_b,RRR_b,DD_WE_b,DD_NS_b,Po(p)_b,POA_b,hour
0,19128,6,65.24,66.29,195.2,31.2,60.0,2.0,0.0,1,...,31.25,27.5,83.0,0.0,0.0,-1.0,0.0,996.39,441.37,11
1,19129,7,72.97,74.12,196.3,31.2,60.0,2.0,0.0,1,...,35.22,27.5,83.0,0.0,0.0,-1.0,0.0,996.39,441.37,11
2,19130,8,65.92,66.98,199.0,31.2,60.0,2.0,0.0,1,...,39.62,27.5,83.0,0.0,0.0,-1.0,0.0,996.39,441.37,11
3,19131,9,72.45,73.67,232.2,31.2,60.0,2.0,0.0,1,...,50.39,27.5,83.0,0.0,0.0,-1.0,0.0,996.39,441.37,11
4,19132,10,60.01,60.98,182.0,31.2,60.0,2.0,0.0,1,...,25.24,27.5,83.0,0.0,0.0,-1.0,0.0,996.39,441.37,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,50268,6,8.44,8.78,55.4,8.0,96.0,2.0,4.0,1,...,4.75,8.5,98.0,2.0,5.0,1.0,0.0,1011.46,959.15,14
2391,50269,7,7.66,7.94,55.2,8.0,96.0,2.0,4.0,1,...,4.74,8.5,98.0,2.0,5.0,1.0,0.0,1011.46,959.15,14
2392,50270,8,8.21,8.45,54.5,8.0,96.0,2.0,4.0,1,...,4.91,8.5,98.0,2.0,5.0,1.0,0.0,1011.46,959.15,14
2393,50271,9,9.55,9.84,58.0,8.0,96.0,2.0,4.0,1,...,5.23,8.5,98.0,2.0,5.0,1.0,0.0,1011.46,959.15,14


In [19]:
features = ['T', 'U', 'Ff',	'RRR', 'DD_WE', 'DD_NS', 'POA', 'Po(p)', 'hour',
            'T_b', 'U_b', 'Ff_b', 'RRR_b', 'DD_WE_b', 'DD_NS_b', 'POA_b', 'Po(p)_b', 'device_id']

X = df[features]
y = df['cc_diff']

X = X.fillna(X.mean())
y = y.fillna(y.mean())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)

In [20]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['sqrt', 'log2', None]
}

gb = GradientBoostingRegressor()

# RandomizedSearchCV setup
gb_random = RandomizedSearchCV(gb, param_distributions=param_distributions, 
                                   n_iter=100, cv=5, verbose=2, random_state=100, 
                                   n_jobs=-1, scoring=make_scorer(mean_squared_error, greater_is_better=False))

# Fit the model (replace X_train and y_train with your training data)
gb_random.fit(X_train, y_train)

# Best parameters
print("Best parameters:", gb_random.best_params_)

# Best score (negative mean squared error)
print("RMSE:", np.sqrt(-gb_random.best_score_))

best_model = gb_random.best_estimator_

y_pred1 = best_model.predict(X_pred)

print("gb:")
print('MAE:', mean_absolute_error(y_pred, y_pred1))
print('MSE:', mean_squared_error(y_pred, y_pred1))
print('RMSE:', np.sqrt(mean_squared_error(y_pred, y_pred1)))
print('R2:', r2_score(y_pred, y_pred1))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 7, 'learning_rate': 0.05}
RMSE: 7.028491174543872
gb:
MAE: 15.12960928241417
MSE: 549.514307752344
RMSE: 23.441721518530674
R2: 0.8126095200534289
[CV] END learning_rate=0.2, max_depth=6, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   0.6s
[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=6, min_samples_split=6, n_estimators=200; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=6, min_samples_split=6, n_estimators=200; total time=   0.3s
[CV] END learning_rate=0.05, max_depth=6, max_features=None, min_samples_leaf=6, min_samples_split=6, n_estimators=400; total time=   1.9s
[CV] END learning_rate=0.05, max_depth=5, max_features=None, min_samples_leaf=1, min_samples_split=8, n_estimato

In [23]:
y_pred_df = pd.DataFrame(y_pred1, columns=['Predicted'])
df_pred['Predicted'] = y_pred_df['Predicted']
df_pred

,id,device_id,active_power,direct_power,charge_capacity,date_time,T,U,Ff,RRR,...,U_b,Ff_b,RRR_b,DD_WE_b,DD_NS_b,Po(p)_b,POA_b,difference,hour,Predicted
0,104983,6,2.31,2.69,1.8,2024-02-23 08:00:00,5.2,95,1,17.0,...,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8,1.789807
1,104984,7,2.38,2.79,1.9,2024-02-23 08:00:00,5.2,95,1,17.0,...,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8,1.804132
2,104985,8,2.37,2.83,1.9,2024-02-23 08:00:00,5.2,95,1,17.0,...,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8,1.974089
3,104986,9,2.44,2.84,2.2,2024-02-23 08:00:00,5.2,95,1,17.0,...,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8,1.936632
4,104987,10,2.32,2.68,1.8,2024-02-23 08:00:00,5.2,95,1,17.0,...,86.0,2.0,3.0,1.0,1.0,1013.05,0.0,0 days 03:00:00,8,1.653518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,106183,6,3.70,4.24,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8,7.130874
251,106184,7,3.80,3.99,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8,7.461271
252,106185,8,3.80,4.19,0.9,2024-03-04 08:00:00,9.6,88,3,0.5,...,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8,7.749842
253,106186,9,4.02,4.39,0.8,2024-03-04 08:00:00,9.6,88,3,0.5,...,76.0,2.0,0.0,-1.0,0.0,1004.72,0.0,0 days 03:00:00,8,9.297994


In [24]:
df_pred[['date_time','cc_diff','Predicted']].to_csv('results.csv')